In [54]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from collections import defaultdict

In [55]:
import nltk
import random
#from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
def read_files():
    d = defaultdict(list)

    parent_dir = '../Data Sets/'

    path_dic = {'B': 'business_s.csv', 'R':'review_m.csv'}
#                ,'C':'checkin.csv', 'T':'tip.csv', 'U':'user.csv'}

    for key in path_dic:
        d[key] = pd.read_csv(parent_dir + path_dic[key]).drop('Unnamed: 0', 1)
    return d
def show():
    sns.despine()
    plt.show()
    
def clean_format(w):
    w = w.lower().replace('.', '').replace(',', '').replace('!', '')
    #.replace('+', '').replace('(', '').replace(')', '')
    return w

In [105]:
# np.random.seed(47)
# parent_dir = '../Data Sets/'
# temp = d['R'].sample(frac = 0.1, replace = False)
# temp.shape
# temp.to_csv(parent_dir + 'review_m.csv')

In [106]:
d = read_files()

/Users/zihaoxu/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [107]:
d['RB'] = d['R'].merge(d['B'], on = 'business_id', how = 'inner')
d['RB'] = d['RB'].dropna(subset = ['is_open'])
d['RB'].rename(columns = {'stars_x' : 'review_star', 'stars_y':'buz_star'}, inplace = True)
d['RB'].shape

(437631, 25)

In [108]:
print(list(d['RB']))

['business_id', 'cool', 'date', 'funny', 'review_id', 'review_star', 'text', 'useful', 'user_id', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood', 'postal_code', 'review_count', 'buz_star', 'state', 'price', 'credit_card']


# Data preparation

In [111]:
df = d['RB']
df.shape

(437631, 25)

In [112]:
# Stem all the words in reviews
ps = PorterStemmer()

df.index = range(len(df))
df['text'] = [' '.join([ps.stem(w) for w in df['text'][i].split()]) for i in range(len(df['text']))]

KeyboardInterrupt: 

In [114]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['state'], train_size = .8, random_state = 47)

# NMF and LDA

In [115]:
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [116]:
# Deafult values and helper function to print the topics
n_features = 5000
n_components = 10
n_top = 15

def print_top_words(model, feature_names, n_top):
    for topic_ind, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_ind
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top - 1:-1]])
        print(message)
    print()

In [117]:
# Extract TF for LDA:
t0 = time()
count_vect = CountVectorizer(max_df = .95, min_df = 3, ngram_range = (1,2), \
                             max_features = n_features, stop_words = 'english')
tf = count_vect.fit_transform(X_train)
print("TF extracted in %0.3fs" % (time() - t0))

# Extract TF-IDF for NMF:
t0 = time()
tfidf_vect = TfidfVectorizer(max_df = .95, min_df = 3, ngram_range = (1,2), \
                             max_features = n_features, stop_words = 'english')
tfidf = tfidf_vect.fit_transform(X_train)
print("TFIDF extracted in %0.3fs" % (time() - t0))

TF extracted in 109.040s
TFIDF extracted in 130.580s


## Fitting the NMF & LDA models

In [119]:
# Fit the NMF model
t0 = time()
nmf = NMF(n_components = 20, random_state = 47, alpha = .1, l1_ratio = .5).fit(tfidf)
print("done in %0.3f" % (time() - t0))
print("\nTopics from NMF model: ")
tfidf_feature_names = tfidf_vect.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top)

done in 157.999

Topics from NMF model: 
Topic 0: good pretty food good pretty good good food good service service good really good service good place place prices price place good overall
Topic 1: like just don car know store work people new make time day going need way
Topic 2: great great service great food food great service great great place service atmosphere place great great atmosphere great experience awesome great job prices time
Topic 3: pizza crust cheese wings pizzas toppings best pizza pepperoni sauce slice pizza place delivery good pizza pizza good great pizza
Topic 4: ice cream ice cream chocolate coffee tea flavors dessert cake cookies vanilla cone cookie sweet waffle
Topic 5: service customer customer service great customer excellent horrible service great excellent customer rude great service worst excellent service manager poor thank
Topic 6: burger fries burgers cheese bun bacon onion rings onion rings burger fries shake sweet potato patty potato potato fries
Topic

In [120]:
# Fit the LDA model
t0 = time()
lda = LatentDirichletAllocation(n_topics = 20, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=47).fit(tf)
print("done in %0.3fs." % (time() - t0))
print_top_words(lda, tfidf_feature_names, n_top)

done in 787.078s.
Topic 0: place great love staff friendly amazing definitely awesome favorite great place recommend love place best sandwiches absolutely
Topic 1: cream ice tea chocolate ice cream cake sweet try delicious flavors like milk love dessert taste
Topic 2: like don just place know really want think good people easy way thing price better
Topic 3: chicken sauce ordered rice fried like good beef pork food soup spicy flavor meat salad
Topic 4: feel hair did job taking like appointment feel like amazing cut salon nails massage happy clean
Topic 5: ve time years make staff sure times care going dr family year make sure ago office
Topic 6: bar place nice area selection good staff location drinks friendly drink like great local little
Topic 7: best vegas ve las las vegas italian bartender far strip amazing town favorite eaten cocktails orange
Topic 8: food great service pizza good atmosphere friendly amazing great service delicious great food service great nice food great place
To

In [121]:
# lda.__dict__
# import inspect
# inspect.getmembers(lda, predicate=inspect.ismethod)


save_model = open("pickled_algos/lda.pickle","wb")
pickle.dump(lda, save_model)
save_model.close()

save_model = open("pickled_algos/nmf.pickle","wb")
pickle.dump(nmf, save_model)
save_model.close()

In [94]:
lda.score(tfidf_vect.fit_transform(X_train))

-178950.47513472088

In [131]:
no_top_documents = 5
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Features of Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print()
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print("doc_index: ", doc_index)
            print(documents[doc_index])
            print()
nmf_W = nmf.transform(tfidf)
nmf_H = nmf.components_
display_topics(nmf_H, nmf_W, tfidf_feature_names, X_train, n_top, no_top_documents)


Features of Topic 0:
good pretty food good pretty good good food good service service good really good service good place place prices price place good overall

doc_index:  243835
Its like walking into a Sopranos set.All those gangsta types.Never mind the pizza is great.Lot of gambling there though. The food is good,just the patrons creep me out.
I never eat in that area but when I do its Nini Daversa

doc_index:  122365
The food is excellent especially the lobster dishes. However when I went there with a friend their menu is all in Chinese and my friend and I could not read the menu because we don't read or speak Chinese. We have asked for an English menu but the server said they didn't have one! ?! So we told him what we wanted and he pretty much did the ordering for us. The dishes were pretty good however I would recommend you go with someone who speaks or read chinese because it would be a challenge to order otherwise.

doc_index:  18410
ive been hearing that this is the breakfast 

KeyError: 285436

In [138]:
lda_W = lda.transform(tf)
lda_H = lda.components_

tf_feature_names = count_vect.get_feature_names()
display_topics(lda_H, lda_W, tf_feature_names, X_train, n_top, no_top_documents)

Features of Topic 0:
place great love staff friendly amazing definitely awesome favorite great place recommend love place best sandwiches absolutely

doc_index:  234320
Undeserving of the current 4+ star rating. Food isn't horrible but isn't anything special. Service was slow and inattentive for a near empty restaurant. Be sure to check your pho order because you may be missing some meats (ex. No tendon)

doc_index:  310093
Great foreign car repair shop, extremely capable, friendly and honest. Definitely professional with my Mercedes A+++

doc_index:  179453
This is an Italian deli that serves Italian food.  I read a few fellow Yelpers reviews and spotted the occasional check-ins which stoked my curiosity to eventually try it for myself.  If you are expecting a nice, polished eatery, this is not it.  But they do have a good selection of cured sausages, prepared sides, and serve a menu of traditional Italian fare.  Including a very mean Lobster Roll.

That was what I ended up ordering b

KeyError: 77440